<a href="https://colab.research.google.com/github/beatlesatani/coding_september/blob/main/web_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install flask-ngrok

In [1]:
!pip install tensorflow_text
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [5]:
from tensorflow import keras

# モデルファイルのパスを指定
model_path = '/drive/MyDrive/sentiment_analysis/bert_model.tf'
#OR USE bert_model.h5も使える


# モデルを読み込む
loaded_model = keras.models.load_model(model_path)


In [ ]:
#bert_model takes 2 inputs(input_ids, attention_masks)

In [6]:
from transformers import BertTokenizer,TFBertModel,BertConfig,TFBertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case = True)

In [7]:
import tensorflow as tf

In [24]:
def mask_inputs_for_bert(tweets, max_len):
  # Encode the tweet using the tokenizer.
  encoded_dict = tokenizer.encode_plus(
            tweets,
            add_special_tokens=True, #add cls and sep
            max_length=max_len,   #pad & truncate all sentences
            pad_to_max_length=True,
            return_attention_mask=True  #construct attention_masks
        )
  input_id = encoded_dict['input_ids']
  attention_mask =encoded_dict['attention_mask']

  # Convert the lists to tensors (vectors) and return
  input_id = tf.convert_to_tensor(input_id)
  attention_mask = tf.convert_to_tensor(attention_mask)

  return input_id, attention_mask

In [25]:
test_sentence = "i am so tired and i don't want to write codes anymore"

In [26]:
inp,mask = mask_inputs_for_bert(test_sentence,75)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [27]:
inp

<tf.Tensor: shape=(75,), dtype=int32, numpy=
array([ 101, 1045, 2572, 2061, 5458, 1998, 1045, 2123, 1005, 1056, 2215,
       2000, 4339, 9537, 4902,  102,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)>

In [33]:
inp = tf.reshape(inp, (1, -1))

In [34]:
inp

<tf.Tensor: shape=(1, 75), dtype=int32, numpy=
array([[ 101, 1045, 2572, 2061, 5458, 1998, 1045, 2123, 1005, 1056, 2215,
        2000, 4339, 9537, 4902,  102,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)>

In [28]:
mask

<tf.Tensor: shape=(75,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>

In [35]:
mask = tf.reshape(mask, (1, -1))

In [37]:
output = loaded_model([inp,mask])
_, prediction = torch.max(output, dim=1)

print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')

ValueError: ignored

In [ ]:
#this is the code from hugging face document
input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
outputs = loaded_model(input_ids)
logits = outputs[0]

ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
loaded_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (Custom>TFBertMainLay  multiple                  109482240 
 er)                                                             
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109484547 (417.65 MB)
Trainable params: 109484547 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template
import pickle
import numpy as np

In [ ]:
from flask import Flask, request, render_template

app = Flask(__name__)

# ホームページ
@app.route('/')
def home():
    return render_template('index.html')

# 感情分析
@app.route('/analyze', methods=['POST'])
def analyze():
    text = request.form['text']

    # BERTモデルを使用して感情を予測するコードを追加

    # 予測結果を取得
    sentiment = "Positive"  # ここに感情分析結果を代入

    return render_template('result.html', text=text, sentiment=sentiment)

if __name__ == '__main__':
    app.run(debug=True)
